In [1]:
###Import Packages###
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image

from sklearn.model_selection import train_test_split
from skimage import io, color

import torch
from torch.utils.data  import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from gensim.models import Word2Vec
import string

import re


In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# basedir = "/content/drive/MyDrive/CSCI334301_ComputerVision/Project/Dataset/"
basedir = "/content/drive/MyDrive/CV_Final_Project_Data/data/"
# basedir = "/content/"

df = pd.read_csv(f'{basedir}annot.csv', low_memory = False)
# Remove rows contains only single period
df = df[df.utf8_string != "."]
# Remove all punctuations
df["utf8_string"] = df["utf8_string"].str.replace('[^\w\s]','',regex=True)
df=df.reset_index(drop=True)
print(df.tail())
df.shape

       Unnamed: 0                   id          image_id  \
80004       80004   0ebbecdc46b78d42_9  0ebbecdc46b78d42   
80005       80005  0ebbecdc46b78d42_10  0ebbecdc46b78d42   
80006       80006  0ebbecdc46b78d42_11  0ebbecdc46b78d42   
80007       80007  0ebbecdc46b78d42_13  0ebbecdc46b78d42   
80008       80008  0ebbecdc46b78d42_14  0ebbecdc46b78d42   

                                   bbox utf8_string  \
80004    [623.38, 443.26, 89.92, 42.09]       SWISS   
80005    [311.92, 227.51, 43.24, 48.17]          90   
80006     [431.72, 384.82, 71.7, 62.85]         100   
80007   [507.16, 273.18, 100.7, 135.01]           7   
80008  [387.77, 149.05, 104.33, 127.75]           8   

                                                  points      area  
80004  [631.07, 443.26, 713.3, 467.11, 707.95, 485.35...   3784.73  
80005  [337.61, 227.51, 355.16, 267.88, 329.93, 275.6...   2082.87  
80006  [451.74, 384.82, 503.42, 426.87, 484.8, 447.67...   4506.35  
80007  [507.16, 273.46, 607.86, 

(80009, 7)

In [ ]:
dir_list = os.listdir('/content/drive/MyDrive/CV_Final_Project_Data/data/train_val_images/train_images')
# convert to numpy
dir_list = np.array(dir_list)
print(dir_list.shape)
# convert image_id to numpy
image_id_list = df.image_id.tolist()
# add .jpg to the end of each element
random_list = [x + '.jpg' for x in image_id_list]
random_list = np.array(random_list)
random_list = np.unique(random_list)
# match image_id with dir_list, delete the rows that do not match
dir_list = np.intersect1d(dir_list, random_list)
random_list.shape
# remove .jpg from dir_list
dir_list = [x[:-4] for x in dir_list]

In [ ]:
df = df[df['image_id'].isin(dir_list)]
df = df.reset_index(drop=True)

In [ ]:
df['image_id']=df['image_id'].astype('str')
import ast
df["bbox"] = df["bbox"].apply(ast.literal_eval)

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
h_list=[]
w_list=[]
for i in range(len(df)): #len(df)
  # print(df['image_id'][i])
  # pth=basedir+'train_val_images/train_images/'+df['image_id'][i]+'.jpg'
  bbox=df['bbox'][i]
  # print(bbox)
  # print(bbox[2],bbox[3])
  h_list.append(bbox[2])
  w_list.append(bbox[3])
print(max(h_list),min(h_list))
print(max(w_list),min(w_list))

In [ ]:
def pad_img(path, bbox=None):
  # read image
  img = Image.open(path)
  img = np.array(img)
  print(img.shape)
  if bbox != None:
    #top_left = [min(bbox[1], bbox[3]), min(bbox[0], bbox[2])]
    #bottom_right  = [max(bbox[1], bbox[3]), max(bbox[0], bbox[2])]
    top_left = [bbox[1], bbox[0]]
    bottom_right = [bbox[1]+bbox[3], bbox[0]+bbox[2]]
    img = img[ top_left[0]:bottom_right[0], top_left[1]:bottom_right[1] ] 
  old_image_height, old_image_width, channels = img.shape
  print(img.shape)

  # create new image of desired size and color (blue) for padding
  new_image_width = 1024   # max height 1023.51  min 2.1
  new_image_height = 1024   # max width 910.98   min 1.9

  color = (0,0,0)
  result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

  result[0:old_image_height, 0:old_image_width] = img

  return result

In [ ]:
imgp = pad_img(f'{basedir}train_val_images/train_images/0ebbecdc46b78d42.jpg', [388, 149, 104, 128]	)
#imgp = pad_img(f'{basedir}train_val_images/train_images/a4ea732cd3d5948a.jpg')
plt.imshow(imgp)
plt.show()

In [ ]:
df

In [ ]:
unique_img = pd.unique(df["image_id"])
# then get one row from each image, and drop the rest
df = df[df["image_id"].isin(unique_img)]
df = df.drop_duplicates(subset="image_id", keep="first")
df = df.reset_index(drop=True)
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv("/content/drive/MyDrive/CV_Final_Project_Data/data/unique_annot.csv")

In [ ]:
# make pandas column item to list
df["bbox"] = df["bbox"].apply(ast.literal_eval)

In [ ]:
# read images from image_id
def read_img(path, bbox=None):
    # read image
    # img = Image.open(path)
    image = Image.open(path).convert('L')
    img = np.array(image)
    if bbox != None:
        for j in bbox:
          j = int(j)
        #top_left = [min(bbox[1], bbox[3]), min(bbox[0], bbox[2])]
        #bottom_right  = [max(bbox[1], bbox[3]), max(bbox[0], bbox[2])]
        # if image type is not jpg, pass
        top_left = [bbox[1], bbox[0]]
        bottom_right = [bbox[1]+bbox[3], bbox[0]+bbox[2]]
        img = img[ top_left[0]:bottom_right[0], top_left[1]:bottom_right[1] ]
        # create new image of desired size and color (blue) for padding
        new_image_width = 1024   # max height 1023.51  min 2.1
        new_image_height = 1024   # max width 910.98   min 1.9

        color = (0,0,0)
        result = np.full((new_image_height,new_image_width, 1), color, dtype=np.uint8)

        result[0:img.shape[0], 0:img.shape[1]] = img
    return Image.fromarray(img)

In [ ]:
# read the images using read_img
def read_images(df, basedir):
    images = []
    for i in range(len(df)):
    # for i in range(2):
        print(f'{basedir}train_val_images/train_images/{df["image_id"][i]}.jpg')
        # make df["bbox"][i] into list of integers
        bbox = df["bbox"][i]
        bbox = [int(i) for i in bbox]
        img = read_img(f'{basedir}train_val_images/train_images/{df["image_id"][i]}.jpg', bbox)
        im1 = img.save(f'/content/drive/MyDrive/CV_Final_Project_Data/data/bbox_images/{df["image_id"][i]}.jpg')
        

In [ ]:
# find the largest bbox

In [ ]:
# pad the images using pad_img
def pad_images(df, basedir):
    images = []
    for i in range(len(df)):
    # for i in range(2):
        print(f'{basedir}train_val_images/train_images/{df["image_id"][i]}.jpg')
        # make df["bbox"][i] into list of integers
        bbox = df["bbox"][i]
        bbox = [int(i) for i in bbox]
        img = pad_img(f'{basedir}train_val_images/train_images/{df["image_id"][i]}.jpg', bbox)
        im1 = Image.fromarray(img)
        im1 = im1.save(f'/content/drive/MyDrive/CV_Final_Project_Data/data/padded_images/{df["image_id"][i]}.jpg')
        # images.append(img)
    return images

In [ ]:
read_images(df, basedir)